In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from tqdm import tqdm
import statsmodels.api as sm

In [ ]:
#excel files are read and a ticker column is added to each sheet

repertoire = "data/raw/stocks" 
dataframes_dict = {}

for fichier in os.listdir(repertoire): 
    chemin_complet = os.path.join(repertoire, fichier)

    if fichier.endswith((".xls", ".xlsx")):  
        print(f"File found : {fichier}")

    try:
        worksheet = pd.read_excel(chemin_complet, sheet_name=None)

        for sheet_name, df in worksheet.items():
            if 'Ticker' not in df.columns:
                df['Ticker'] = sheet_name  
                print(f"Ticker added to {sheet_name}")
                dataframes_dict[f"{fichier}_{sheet_name}"] = df
            else:
                print(f"Column 'Ticker' already existed in {fichier}")

    except Exception as e:
        print(f"Error during the process {fichier} : {e}")

In [ ]:
#variables are renamed and split by frequency
monthly_list = []
yearly_list = []
quarterly_list = []

daily_data_list = [] #we create multiple lists because the start dates differ
daily_tot_return_list = []
daily_askbid_list = []

gics_list = []

for name, df in dataframes_dict.items():
    rename_map = {
        df.columns.values[0]: "Date.1",
        df.columns.values[1]: "Total_Assets",
        df.columns.values[2]: "Common_Equity",
        df.columns.values[3]: "Cash_And_Investments",
        df.columns.values[4]: "R&D_Expenses",
        df.columns.values[5]: "Inventories",
        df.columns.values[6]: "Dividends_Paid",
        df.columns.values[7]: "Gross_Fixed_Assets",
        df.columns.values[8]: "Income_Before_Extra_Items",
        df.columns.values[9]: "Sales_Revenue",
        df.columns.values[10]: "Depreciation_Amortization",

        df.columns.values[12]: "Date.2",
        df.columns.values[13]: "Mkt_Cap_Yearly",
        df.columns.values[14]: "Shares_Outstanding_Yearly",
        df.columns.values[15]: "Long_Term_Debt",

        df.columns.values[17]: "Date.3",
        df.columns.values[18]: "Net_Income",

        df.columns.values[20]: "Date.4",
        df.columns.values[21]: "Shares_Outstanding_Monthly",
        df.columns.values[22]: "Mkt_Cap_Monthly",

        df.columns.values[24]: "Date.5",
        df.columns.values[25]: "Px_Last",
        df.columns.values[26]: "Shares_Outstanding_Daily",
        df.columns.values[27]: "Volume",

        df.columns.values[29]: "Date.6",
        df.columns.values[30]: "Total_Return",

        df.columns.values[32]: "Date.7",
        df.columns.values[33]: "Px_Ask",
        df.columns.values[34]: "Px_Bid",

        df.columns.values[42]: "Industry",
        df.columns.values[43]: "Sector",

    }

    df.rename(columns=rename_map, inplace=True)

    try:
        df_yearly = df[[
            "Date.1", "Total_Assets", "Common_Equity", "Cash_And_Investments",
            "R&D_Expenses", "Inventories", "Dividends_Paid", "Gross_Fixed_Assets",
            "Income_Before_Extra_Items", "Sales_Revenue", "Depreciation_Amortization",
            "Mkt_Cap_Yearly", "Shares_Outstanding_Yearly", "Long_Term_Debt"
        ]].copy()
        
        df_quarterly = df[["Date.3", "Net_Income"]].copy()
       
        df_monthly = df[[
            "Date.4", "Shares_Outstanding_Monthly", "Mkt_Cap_Monthly" 
        ]].copy()

        df_daily_data = df[[
            "Date.5", "Px_Last", "Shares_Outstanding_Daily", "Volume"
        ]].copy()

        df_daily_tot_return = df[[
            "Date.6", "Total_Return"
        ]].copy()

        df_daily_askbid= df[[
            "Date.7", "Px_Ask", "Px_Bid"
        ]].copy()

        df_gics = df[[
            "Industry", "Sector"
        ]].copy()
        
        df_yearly['Ticker'] = df['Ticker'].iloc[0]
        df_quarterly['Ticker'] = df['Ticker'].iloc[0]
        df_monthly['Ticker'] = df['Ticker'].iloc[0]
        df_daily_data['Ticker'] = df['Ticker'].iloc[0]
        df_daily_tot_return['Ticker'] = df['Ticker'].iloc[0]
        df_daily_askbid['Ticker'] = df['Ticker'].iloc[0]
        df_gics['Ticker'] = df['Ticker'].iloc[0]

        yearly_list.append(df_yearly)
        quarterly_list.append(df_quarterly)
        monthly_list.append(df_monthly)
        daily_data_list.append(df_daily_data)
        daily_tot_return_list.append(df_daily_tot_return)
        daily_askbid_list.append(df_daily_askbid)
        gics_list.append(df_gics)


        print(f"{name} : successful yearly / monthly / quarterly / gics split")

    except Exception as e:
        print(f"{name} : error during split : {e}")

yearly_df = pd.concat(yearly_list, ignore_index=True)
quarterly_df = pd.concat(quarterly_list, ignore_index=True)
monthly_df = pd.concat(monthly_list, ignore_index=True)

daily_data_df = pd.concat(daily_data_list, ignore_index=True)
daily_tot_return_df = pd.concat(daily_tot_return_list, ignore_index=True)
daily_askbid_df = pd.concat(daily_askbid_list, ignore_index=True)

gics_df = pd.concat(gics_list, ignore_index=True)


In [ ]:
lists = {
    "yearly": yearly_df,
    "quarterly": quarterly_df,
    "monthly": monthly_df,
    "daily_tot_return": daily_tot_return_df,
    "daily_data": daily_data_df,
    "daily_askbid": daily_askbid_df,
}

for name, df in lists.items():
    #we rename columns starting with "Date.xxx" to "Date"
    df.rename(columns={col: "Date" for col in df.columns if col.startswith("Date")}, inplace=True)
    
    #we convert "Date" column to datetime
    if "Date" in df.columns:
        df["Date"] = pd.to_datetime(df["Date"], errors='coerce')
    
    #we convert all other columns to float64 (excluding "Date" and "Ticker")
    for col in df.columns:
        if col not in ["Date", "Ticker"]:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype("float64")

    #display column information for verification
    print(f"--- {name.capitalize()} DataFrame ---")
    print(f"  - Colonnes et Types:")
    for col in df.columns:
        print(f"    - {col}: {df[col].dtype}")
    print(f"  - Nombre de lignes: {df.shape[0]}")
    print(f"  - Nombre de colonnes: {df.shape[1]}")
    print("-" * 50)

In [ ]:
#NaN Cells
"""
We remove rows where the date column is NaN. When splitting the data by frequency, each row is assigned a ticker.
However, because daily data have more rows than other frequencies, the ticker is excessively duplicated in the lower-frequency 
DataFrames (monthly, quarterly, yearly), leading to rows that are mostly empty.
Dropping rows without a date removes these redundancies without any loss of actual data.
"""


lists["yearly"] = yearly_df.dropna(subset=["Date"])
lists["quarterly"] = quarterly_df.dropna(subset=["Date"])
lists["monthly"] = monthly_df.dropna(subset=["Date"])
lists["daily_data"] = daily_data_df.dropna(subset=["Date"])
lists["daily_tot_return"] = daily_tot_return_df.dropna(subset=["Date"])
lists["daily_askbid"] = daily_askbid_df.dropna(subset=["Date"])

all_nan_matrices = {}

for name, df in lists.items():  
    if "Ticker" in df.columns:
        cols = [col for col in df.columns if col != "Ticker"]
        nan_matrix = (
            df.groupby("Ticker")[cols]
              .apply(lambda g: g.isna().mean() * 100)
              .reset_index()
        )
        all_nan_matrices[name] = nan_matrix
        print(f"\n % Pourcentage de NaN pour {name} :\n", nan_matrix)

In [ ]:
#NaN handling using SimpleImputer
from sklearn.impute import SimpleImputer

#create an imputer to replace NaNs with the mean
imputer = SimpleImputer(strategy="mean")

for name, df in lists.items():
    cols_to_impute = [col for col in df.columns if col not in ["Date", "Ticker"]]
    df.loc[:, cols_to_impute] = df.groupby("Ticker")[cols_to_impute].transform(lambda x: x.fillna(x.mean()))

In [ ]:
yearly_df = lists["yearly"]
quarterly_df = lists["quarterly"]
monthly_df = lists["monthly"]
daily_data_df = lists["daily_data"]
daily_tot_return_df = lists["daily_tot_return"]
daily_askbid_df = lists["daily_askbid"]

In [ ]:
def convert_to_monthly_period(df, date_column="Date"):
    """Convertit la colonne Date en période mensuelle si nécessaire"""
    if pd.api.types.is_datetime64_any_dtype(df[date_column]):
        df[date_column] = df[date_column].dt.to_period("M")
        print(f"{date_column} converted to monthly period")
    else:
        print(f"{date_column} already in monthly period")
    return df

In [ ]:
#yearly computations    

#mkt cap and shares outstanding were extracted with a BDH formula and are expressed in millions so we have to multiply by 10^6
cols_to_scale = ["Mkt_Cap_Yearly", "Shares_Outstanding_Yearly", "Long_Term_Debt"]
yearly_df[cols_to_scale] = yearly_df[cols_to_scale] * 1e6

daily_data_df["Shares_Outstanding_Daily"] = daily_data_df["Shares_Outstanding_Daily"] * 1e6

II. 1. Yearly and quarterly computations 

In [ ]:
## 1. On mensualise les data yearly 

# --- 1.1 Création différents types de colonnes
cols_stocks = ["Total_Assets", "Common_Equity", "Cash_And_Investments", "Inventories", "Gross_Fixed_Assets", "Mkt_Cap_Yearly", 
"Shares_Outstanding_Yearly", "Long_Term_Debt"]
cols_flows = ["R&D_Expenses", "Dividends_Paid", "Sales_Revenue", "Depreciation_Amortization", "Income_Before_Extra_Items"]
rows_monthly = []

# --- 1.2 Calcul rendement que l'on mensualisera après 
yearly_df["dy"] = yearly_df["Dividends_Paid"] / yearly_df["Mkt_Cap_Yearly"]

# --- 1.3 Mensualisation des données : les stocks sont identiques sur 12 mois, les flux sont divisés par 12 et le rendement est mensualisé
for index, row in yearly_df.iterrows():
    year = row["Date"].year
    ticker = row["Ticker"]

    for month in range(1, 13):
        new_row = {"Ticker": ticker, "Date": pd.Period(year=year, month=month, freq="M")}

        for col in cols_stocks:
            new_row[col] = row[col]

        for col in cols_flows:
            new_row[col] = row[col] / 12 if pd.notnull(row[col]) else None

        dy_annual = row["dy"]
        new_row["dy"] = (1 + dy_annual) ** (1/12) - 1 if pd.notnull(dy_annual) else None

        rows_monthly.append(new_row)

# 4. Résultat final
monthly_from_yearly = pd.DataFrame(rows_monthly)

In [ ]:
## 2. Computations

# --- 2.1 Calculs basiques
#asset growth (agr)
monthly_from_yearly["agr"] = (
    monthly_from_yearly["Total_Assets"] - monthly_from_yearly.groupby("Ticker")["Total_Assets"].shift(12)
) / monthly_from_yearly.groupby("Ticker")["Total_Assets"].shift(12)


#cash productivity (cashpr)
monthly_from_yearly["cashpr"] = (
    monthly_from_yearly["Mkt_Cap_Yearly"] + monthly_from_yearly["Long_Term_Debt"] - monthly_from_yearly["Total_Assets"]
) / monthly_from_yearly["Cash_And_Investments"]


#change in inventory (chinv)
monthly_from_yearly["chinv"] = (
    monthly_from_yearly["Inventories"] - monthly_from_yearly.groupby("Ticker")["Inventories"].shift(12)
) / monthly_from_yearly["Total_Assets"]

#change in shares outstanding (chsh)   
monthly_from_yearly["chsh"] = (
    monthly_from_yearly["Shares_Outstanding_Yearly"] - monthly_from_yearly.groupby("Ticker")["Shares_Outstanding_Yearly"].shift(12)
) / monthly_from_yearly.groupby("Ticker")["Shares_Outstanding_Yearly"].shift(12)

#depreciation / Gross Fixed Assets (depr)
monthly_from_yearly["depr"] = (
    monthly_from_yearly["Depreciation_Amortization"] /
    monthly_from_yearly["Gross_Fixed_Assets"]
)

#earnings to Price (ep)
monthly_from_yearly["ep"] = (
    monthly_from_yearly["Income_Before_Extra_Items"] /
    monthly_from_yearly["Mkt_Cap_Yearly"]
)

#investment to assets (invest)
monthly_from_yearly["invest"] = (
    monthly_from_yearly.groupby("Ticker")["Gross_Fixed_Assets"].transform(lambda x: x - x.shift(12)) +
    monthly_from_yearly.groupby("Ticker")["Inventories"].transform(lambda x: x - x.shift(12))
) / monthly_from_yearly.groupby("Ticker")["Total_Assets"].transform(lambda x: x.shift(12))


#R&D to Market Value of Equity (rd_mve)
monthly_from_yearly["rd_mve"] = (
    monthly_from_yearly["R&D_Expenses"] /
    monthly_from_yearly["Mkt_Cap_Yearly"]
)


#sales to Price (sp)
monthly_from_yearly["sp"] = (
    monthly_from_yearly["Sales_Revenue"] /
    monthly_from_yearly["Mkt_Cap_Yearly"]
)

# --- 2.2 On filtre les données on ne garde les covariates à partir de 1990-12
yearly_cov = monthly_from_yearly[["Ticker", "Date", "agr", "cashpr", "chinv", "chsh", "depr", "dy", "ep", "invest", "rd_mve", "sp"]].copy()
yearly_cov = yearly_cov[yearly_cov["Date"] >= pd.Period("1990-12", freq="M")]
yearly_cov = yearly_cov.reset_index(drop=True)

In [ ]:
#quarterly computations
quarterly_df["delta_income"] = (
    quarterly_df["Net_Income"] - quarterly_df["Net_Income"].shift(1)
)

quarterly_df["direction"] = np.sign(quarterly_df["delta_income"])

def compute_nincr(direction_series):
    nincr = []
    count = 0
    prev = 0
    for d in direction_series:
        if d == prev and d != 0:
            count += 1
        elif d != 0:
            count = 1
        else:
            count = 0
        capped_count = min(count, 8)
        nincr.append(count * d if d != 0 else 0)
        prev = d if d != 0 else prev
    return nincr

quarterly_df["nincr"] = (
    quarterly_df.groupby("Ticker")["direction"].transform(compute_nincr)
)


II.2. Monthly computations 

Each subsections correspond to a variable family : momentums, illiquidity, risks, size 

In [ ]:
tot_return_raw = daily_tot_return_df[["Date", "Ticker", "Total_Return"]].copy() #we'll need it to compute weekly returns for beta afterwards 

monthly_data = {
    "daily_tot_return_df": daily_tot_return_df,
    "monthly_df": monthly_df,
    "daily_data_df": daily_data_df,
    "daily_askbid_df": daily_askbid_df,
}

#date to monthly period
for name, df in monthly_data.items():
    if pd.api.types.is_datetime64_any_dtype(df["Date"]):
        df["Date"] = df["Date"].dt.to_period("M")
        print(f"{name} : Date converted to monthly period")
    else:
        print(f"{name} : Date already converted")

In [ ]:
## 1. Momentum Variables

#daily return computation
df_daily_return = daily_tot_return_df[["Date", "Ticker", "Total_Return"]].copy()

df_daily_return["daily_return"] = (
    df_daily_return.groupby("Ticker")["Total_Return"].pct_change()
)

df_daily_return = convert_to_monthly_period(df_daily_return)

# --- 1.1 Momentum computation
def compute_momentum(df, lag_start, lag_end):
    return (1 + df.shift(lag_start)).rolling(lag_end - lag_start + 1, min_periods=lag_end - lag_start + 1).apply(np.prod, raw=True) - 1

df_monthly_return = (
    df_daily_return.groupby(["Ticker", "Date"])["daily_return"]
    .apply(lambda x: (1 + x).prod() - 1)
    .reset_index()
    .rename(columns={"daily_return": "monthly_return"})
)

df_momentum = df_monthly_return.copy()
df_momentum["mom1m"] = df_momentum.groupby("Ticker")["monthly_return"].shift(1)
df_momentum["mom6m"] = df_momentum.groupby("Ticker")["monthly_return"].transform(lambda x: compute_momentum(x, 2, 6))
df_momentum["mom12m"] = df_momentum.groupby("Ticker")["monthly_return"].transform(lambda x: compute_momentum(x, 2, 12))
df_momentum["mom36m"] = df_momentum.groupby("Ticker")["monthly_return"].transform(lambda x: compute_momentum(x, 14, 37))
df_momentum["chmom"] = (
    df_momentum.groupby("Ticker")["monthly_return"].transform(lambda x: compute_momentum(x, 1, 6)) -
    df_momentum.groupby("Ticker")["monthly_return"].transform(lambda x: compute_momentum(x, 7, 12))
)

# --- 1.2 Maximum Return (maxret)
maxret_df = (
    df_daily_return.groupby(["Ticker", "Date"])["daily_return"]
    .max()
    .groupby(level=0)
    .shift(1)
    .reset_index()
    .rename(columns={"daily_return": "maxret"})
)

In [ ]:
## 2. Liquidity Variables

# --- 2.1 Dollar Volume (dolvol_lag2)

df_dolvol = daily_data_df[["Date", "Ticker", "Volume", "Px_Last"]].copy()
df_dolvol["dolvol"] = df_dolvol["Volume"] * df_dolvol["Px_Last"]
df_dolvol = convert_to_monthly_period(df_dolvol)

df_dolvol = (
    df_dolvol.groupby(["Ticker", "Date"])["dolvol"]
    .sum()
    .reset_index(name="dolvol_sum")
)

df_dolvol["dolvol_monthly"] = np.log(df_dolvol["dolvol_sum"])
df_dolvol["dolvol_lag2"] = df_dolvol.groupby("Ticker")["dolvol_monthly"].shift(2)


# --- 2.2 Bid-Ask Spread (mean_baspread)

df_baspread = daily_askbid_df[["Date", "Ticker", "Px_Ask", "Px_Bid"]].copy()
df_baspread["baspread"] = (
    (df_baspread["Px_Ask"] - df_baspread["Px_Bid"]) /
    ((df_baspread["Px_Ask"] + df_baspread["Px_Bid"]) / 2)
)
df_baspread = convert_to_monthly_period(df_baspread)

df_baspread = (
    df_baspread
    .groupby(["Ticker", "Date"], as_index=False)["baspread"]
    .mean()
)

# --- 2.3 Illiquidity (illiq)

df_ill = daily_data_df[["Date", "Ticker", "Volume", "Px_Last"]].copy()
df_ill["daily_return"] = df_ill.groupby("Ticker")["Px_Last"].pct_change()
df_ill["abs_return"] = df_ill["daily_return"].abs()
df_ill["dolvol"] = df_ill["Volume"] * df_ill["Px_Last"]
df_ill["ill_daily"] = df_ill["abs_return"] / df_ill["dolvol"]
df_ill= convert_to_monthly_period(df_ill)

df_illiq = (
    df_ill.groupby(["Ticker", "Date"])["ill_daily"]
    .mean()
    .reset_index()
    .rename(columns={"ill_daily": "illiq"})
)


# --- 2.4 Share Turnover (turn)

df_turn = daily_data_df[["Date", "Ticker", "Volume"]].copy()
df_turn = convert_to_monthly_period(df_turn)

# Moyenne quotidienne du volume par mois
df_turn["mean_volume"] = df_turn.groupby(["Ticker", "Date"])["Volume"].transform("mean")
df_turn = df_turn.drop_duplicates(subset=["Ticker", "Date"])[["Ticker", "Date", "mean_volume"]]

df_turn = df_turn.merge(
    monthly_df[["Ticker", "Date", "Shares_Outstanding_Monthly"]],
    on=["Ticker", "Date"],
    how="left"
)

df_turn["turn"] = (
    df_turn.groupby("Ticker")["mean_volume"]
    .transform(lambda x: x.rolling(3, min_periods=3).mean())
    / df_turn["Shares_Outstanding_Monthly"]
)

# --- 2.5 Share Turnover Volatility (stdturn)

df_stdturn = daily_data_df[["Date", "Ticker", "Volume", "Shares_Outstanding_Daily"]].copy()
df_stdturn = convert_to_monthly_period(df_stdturn)
df_stdturn["daily_turn"] = df_stdturn["Volume"] / df_stdturn["Shares_Outstanding_Daily"]

df_stdturn = (
    df_stdturn.groupby(["Ticker", "Date"])["daily_turn"]
    .std()
    .reset_index()
    .rename(columns={"daily_turn": "stdturn"})
)

In [ ]:
## 3. Size and Valuation Variables

df_size = monthly_df[["Ticker", "Date", "Mkt_Cap_Monthly"]].copy()
df_size["mvel1"] = (
    df_size.groupby("Ticker")["Mkt_Cap_Monthly"]
    .transform(lambda x: np.log(x).shift(1))
)

In [ ]:
## 4. Industry Variables

# --- 4.1 Merge industry info
df_industry = gics_df.drop_duplicates(subset=["Ticker"])[["Ticker", "Sector"]]

df_indmom = df_monthly_return[["Ticker", "Date", "monthly_return"]].copy()
df_indmom = df_indmom.merge(df_industry, on="Ticker", how="left")
 
 # --- 4.2 Compute 12-month momentum per firm
df_indmom["ret_12m"] = (
    df_indmom.groupby("Ticker")["monthly_return"]
    .transform(lambda x: (1 + x.shift(1)).rolling(12, min_periods=12).apply(np.prod, raw=True) - 1)
)

# --- 4.3 Average return per Industry × Date
df_indret = (
    df_indmom.groupby(["Date", "Sector"])["ret_12m"]
    .mean()
    .reset_index()
    .rename(columns={"ret_12m": "indmom"})
)

df_indmom = df_indmom.merge(df_indret, on=["Date", "Sector"], how="left")

In [ ]:
## 5. Excess Return Computation

df_rf_raw = pd.read_excel("data/raw/rf rate/rf Fama&French.xlsx")

# Use only date + risk-free column (assumed to be column 5)
df_rf = df_rf_raw.iloc[:, [0, 4]].copy()
df_rf.columns = ["Date", "rf"]

# Clean numeric format (remove commas, convert to float)
df_rf["rf"] = (
    df_rf["rf"]
    .astype(str)
    .str.replace(",", "")
    .where(lambda x: x.str.match(r"^-?\d+(\.\d+)?$", na=False))
    .astype(float)
)

df_rf["rf"] = df_rf["rf"] / 100

# Convert date to Period(M)
df_rf["Date"] = pd.to_datetime(df_rf["Date"], format="%Y%m", errors="coerce").dt.to_period("M")
df_rf = df_rf.dropna(subset=["Date", "rf"])

# --- 5.2 Merge with monthly returns

df_excess = df_monthly_return.merge(df_rf, on="Date", how="left")
df_excess["excess_return"] = df_excess["monthly_return"] - df_excess["rf"]

# Final clean DataFrame
df_excess_return = df_excess[["Date", "Ticker", "excess_return"]]

df_excess_return = df_excess_return[df_excess_return["Date"] >= pd.Period("1990-12", freq="M")]
df_excess_return.sort_values(["Ticker", "Date"], inplace=True)


In [ ]:
## 6. Risk and Volatility Variables

#return volatility (retvol)
retvol_df = (
    df_daily_return
    .groupby(["Ticker", "Date"])["daily_return"]
    .std()
    .groupby(level=0)  
    .shift(1)          
    .reset_index()
    .rename(columns={"daily_return": "retvol"})
)

In [ ]:
## 7. Beta Computation

# --- 7.1 Weekly returns computation
tot_return_raw = tot_return_raw.set_index("Date")
weekly_prices = tot_return_raw.groupby("Ticker")["Total_Return"].resample("W").last().reset_index()

weekly_prices["weekly_return"] = (
    (weekly_prices["Total_Return"] - weekly_prices.groupby("Ticker")["Total_Return"].transform("shift", 1))
    / weekly_prices.groupby("Ticker")["Total_Return"].transform("shift", 1)
)   

pivot = weekly_prices.pivot(index="Date", columns="Ticker", values="weekly_return") #transposée : 1 ligne par date 
pivot["Market"] = pivot.mean(axis=1, skipna=True) #moyenne des returns par semaine

# --- 7.2 Linear Regression

from tqdm import tqdm  #since the computation is long, we use tqdm to display a progress bar
import statsmodels.api as sm

results = []

for ticker in tqdm(pivot.columns.drop("Market")):
    #we loop over each month, starting 3 years after the first date; freq="ME" since we move month by month
    for current_month in pd.date_range(start=pivot.index.min() + pd.DateOffset(years=3), 
                                       end=pivot.index.max(), freq="ME"):

        #current_month is already a timestamp
        end_date = current_month - pd.DateOffset(months=1)
        start_date = end_date - pd.DateOffset(years=3)

        #extract weekly data for this 3-year window, containing the stock and market returns
        try:
            window = pivot.loc[start_date:end_date, [ticker, "Market"]].dropna()
        except KeyError:
            #fallback if the date does not exist exactly
            window = pivot.loc[(pivot.index >= start_date) & (pivot.index <= end_date), [ticker, "Market"]].dropna()

        if len(window) >= 156:  #156 weeks = 3 years
            X = sm.add_constant(window["Market"])  #explicative variable + constant (alpha)
            y = window[ticker]  #dependent variable

            model = sm.OLS(y, X).fit()  #linear regression with statsmodels
            beta = model.params["Market"]  #slope = beta
            alpha = model.params["const"]  #intercept = alpha
            r_squared = model.rsquared  
            p_value = model.pvalues["Market"] #pvalue of beta

            results.append({
                "Ticker": ticker,
                "Date": current_month.to_period("M"),
                "beta": beta,
                "Alpha": alpha,
                "R2": r_squared,
                "p_value": p_value
            })

# final DataFrame + B²
beta_df = pd.DataFrame(results)
beta_df["beta_squared"] = beta_df["beta"]**2


In [ ]:
## 8. Idiovol Computation

pivot["Market"] = pivot.mean(axis=1, skipna=True)

idio_vol_results = []

for ticker in tqdm(pivot.columns.drop("Market")):
    for current_month in pd.date_range(start=pivot.index.min() + pd.DateOffset(years=3),
                                       end=pivot.index.max(), freq="ME"):

        end_date = current_month
        start_date = end_date - pd.DateOffset(years=3)

        try:
            window = pivot.loc[start_date:end_date, [ticker, "Market"]].dropna()
        except KeyError:
            window = pivot.loc[(pivot.index >= start_date) & 
                                      (pivot.index <= end_date), [ticker, "Market"]].dropna()

        if len(window) >= 52:  # At least one year of data
            X = sm.add_constant(window["Market"])
            y = window[ticker]
            model = sm.OLS(y, X).fit()
            residuals = model.resid
            idio_std = np.std(residuals)

            idio_vol_results.append({
                "Ticker": ticker,
                "Date": current_month.to_period("M"),
                "idiovol": idio_std
            })

idio_vol_df = pd.DataFrame(idio_vol_results)

III. Standarisation and mensualisation des variables 

In [ ]:

quarterly_df = convert_to_monthly_period(quarterly_df)

# Étape 2 : création d'une timeline mensuelle complète
timeline = pd.period_range(start="1990-01", end="2020-12", freq="M") #génère tout les mois de la période
monthly_rows = []

# Étape 3 : itération sur les dates trimestrielles
for i in range(len(quarterly_df)): #on lit une ligne une à une
    base_period = quarterly_df.iloc[i]["Date"] #on récupère la date trimestrielle
    value = quarterly_df.iloc[i]["nincr"] #on récupère la valeur de nincr
    ticker = quarterly_df.iloc[i]["Ticker"] #on récupère le ticker

    # Propager sur 3 mois : base, base+1, base+2
    for offset in range(3): #on propage sur 3 mois  
        target_period = base_period + offset #on ajoute l'offset à la date trimestrielle
        if target_period in timeline: #on vérifie si la date est dans la timeline
            monthly_rows.append({ #on ajoute la ligne au DataFrame
                "Ticker": ticker,
                "Date": target_period,  # date de fin de mois
                "nincr": value #on ajoute la valeur de nincr
            })

#df final mensualisé 
monthly_nincr_df = pd.DataFrame(monthly_rows)
monthly_nincr_df = monthly_nincr_df[monthly_nincr_df["Date"] >= pd.Period("1990-12", freq="M")]
monthly_nincr_df.sort_values(["Ticker", "Date"], inplace=True)


In [ ]:
#2. Merging the monthly dataframes
monthly_covariates = monthly_df[["Ticker", "Date"]].copy()
monthly_covariates = monthly_covariates[monthly_covariates["Date"] >= pd.Period("1990-12", freq="M")]

def prepare_df(df):
    df["Date"] = df["Date"].astype("period[M]")
    df = df[df["Date"] >= pd.Period("1990-12", freq="M")].copy()
    return df.reset_index(drop=True)
df_dolvol = prepare_df(df_dolvol)
maxret_df = prepare_df(maxret_df)
retvol_df = prepare_df(retvol_df)
df_momentum = prepare_df(df_momentum)
df_turn = prepare_df(df_turn)
df_indmom = prepare_df(df_indmom)
df_baspread = prepare_df(df_baspread)
df_illiq = prepare_df(df_illiq)
df_stdturn = prepare_df(df_stdturn)
beta_df = prepare_df(beta_df)
idio_vol_df = prepare_df(idio_vol_df)


In [ ]:
merge_tasks = [
    {"df": df_dolvol, "col": "dolvol_lag2"},
    {"df": maxret_df, "col": "maxret"},
    {"df": retvol_df, "col": "retvol"},
    {"df": df_momentum, "col": "mom36m"},
    {"df": df_momentum, "col": "mom12m"},
    {"df": df_momentum, "col": "mom6m"},
    {"df": df_momentum, "col": "mom1m"},
    {"df": df_momentum, "col": "chmom"},
    {"df": df_turn, "col": "turn"},
    {"df": df_indmom, "col": "indmom"},
    {"df": df_baspread, "col": "baspread"},
    {"df": df_illiq, "col": "illiq"},
    {"df": df_stdturn, "col": "stdturn"},
    {"df": beta_df, "col": "beta"},
    {"df": beta_df, "col": "beta_squared"},
    {"df": idio_vol_df, "col": "idiovol"},
    {"df": df_size, "col": "mvel1"},
]

for task in merge_tasks:
    df_to_merge = task["df"]
    col = task["col"]

    df_to_merge = df_to_merge[df_to_merge["Date"] >= pd.Period("1990-12", freq="M")].copy()
    df_to_merge.reset_index(drop=True, inplace=True)

    if col not in monthly_covariates.columns:
        print(f"{col} merged into monthly_df")
        monthly_covariates = monthly_covariates.merge(
            df_to_merge[["Ticker", "Date", col]],
            how="left",
            on=["Ticker", "Date"]
        )
    else:
        print(f"{col} already in monthly_covariates — skipping.") 

In [ ]:
# Merge monthly_from_yearly
# Merge avec yearly_cov

for col in yearly_cov.columns:
    if col not in monthly_covariates.columns:
        monthly_covariates = pd.merge(monthly_covariates, yearly_cov, on=["Ticker", "Date"], how="left")
        print(f"{col} merged into monthly_covariates")
    else:
        print(f"{col} already in monthly_covariates — skipping.")

# Merge monthly_nincr
for col in monthly_nincr_df.columns:
    if col not in ["Ticker", "Date"]:
        if col not in monthly_covariates.columns:
            monthly_covariates = pd.merge(monthly_covariates, monthly_nincr_df, on=["Ticker", "Date"], how="left")
            print(f"{col} merged into monthly_covariates")
        else:
            print(f"{col} already in monthly_covariates — skipping.")

#merge avec excess returns 
for col in df_excess_return.columns:
    if col not in ["Ticker", "Date"]:
        if col not in monthly_covariates.columns:
            monthly_covariates = pd.merge(monthly_covariates, df_excess_return, on=["Ticker", "Date"], how="left")
            print(f"{col} merged into monthly_covariates")
        else:
            print(f"{col} already in monthly_covariates — skipping.")

In [ ]:
#Liste des covariables à normaliser
covariates = ["dolvol_lag2", "maxret", "retvol", "mom36m", "mom12m", "mom6m", "mom1m", "chmom", "turn", 
"indmom", "baspread", "illiq", "stdturn", "beta", "beta_squared", "idiovol", "mvel1"]

for cov in covariates:
    monthly_covariates[f'{cov}_norm'] = np.nan

for date_key in monthly_covariates['Date'].unique():
    group = monthly_covariates[monthly_covariates['Date'] == date_key]
    idx = group.index
    for cov in covariates:
        temp = group[[cov]].sort_values(by=cov).dropna()
        n = len(temp)
        if n == 1:
            scores = [0]
        else:
            scores = 2 * np.arange(n) / (n - 1) - 1
        monthly_covariates.loc[temp.index, f'{cov}_norm'] = scores

df_final = monthly_covariates[['Date', 'Ticker'] + [f'{cov}_norm' for cov in covariates]]

In [ ]:
df_final.to_excel("data/processed/df_final.xlsx")

In [1]:
pip install git-filter-repo


Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
